<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/VoxelCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q mlflow
!databricks configure --host https://community.cloud.databricks.com/

Username: gfesta24@gmail.com
Password: 
Repeat for confirmation: 


In [8]:
import sys
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn 
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
import numpy as np
from sklearn.model_selection import ParameterGrid

In [4]:
train_df = np.load('/content/drive/Shareddrives/Datasets SEFAI/voxel_train_dataset.npz')
test_df = np.load('/content/drive/Shareddrives/Datasets SEFAI/voxel_test_dataset.npz')

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X_array, Y_array, transform=None):
        self.X = X_array
        self.Y = Y_array
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        label = self.Y[index] 

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [6]:
import mlflow
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/gfesta24@gmail.com/Voxels3DCNN")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2211619870750348', creation_time=1686564463983, experiment_id='2211619870750348', last_update_time=1686564463983, lifecycle_stage='active', name='/Users/gfesta24@gmail.com/Voxels3DCNN', tags={'mlflow.experiment.sourceName': '/Users/gfesta24@gmail.com/Voxels3DCNN',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'gfesta24@gmail.com',
 'mlflow.ownerId': '1923923806180228'}>

In [7]:
X_train= train_df['data']
X_test = test_df['data']
y_train = train_df['labels']
y_test = test_df['labels']

X_full = np.concatenate((X_train, X_test), axis=0)
y_full = np.concatenate((y_train, y_test), axis=0)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.33, random_state=42, stratify=y_full)

train_dataset = CustomDataset(X_train, y_train, transform=transforms.ToTensor())

test_dataset = CustomDataset(X_test, y_test, transform=transforms.ToTensor())

batch_size=64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [10]:
def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, dim=1)
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc, n_correct;

In [11]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        
        self.conv1 = nn.Conv3d(1, 32, kernel_size=(5, 5, 3), stride=1, padding=(2, 2, 1))
        self.batchnorm1 = nn.BatchNorm3d(32)
        self.maxpool1 = nn.MaxPool3d(kernel_size=(3, 3, 2), stride=(2, 2, 2))
        
        self.conv2 = nn.Conv3d(32, 32, kernel_size=(3, 3, 3), stride=1, padding=0)
        self.batchnorm2 = nn.BatchNorm3d(32)
        self.maxpool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(2560, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.batchnorm1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.batchnorm2(x)
        x = self.maxpool2(x)


        #print(x.shape)
        x = self.flatten(x)
        #print(x.shape)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x




In [13]:
mlflow.end_run()

In [ ]:
param_grid = {'batch_size': [64, 128, 256, 512],'lr': [0.01, 0.001, 0.0001], "momentum":[0.7, 0.8, 0.9], "decay":[0.001, 0.01, 0.1]}
expanded_grid = ParameterGrid(param_grid)
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name("/Users/gfesta24@gmail.com/Voxels3DCNN")


for i in range(len(expanded_grid)):
  runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id], filter_string=" and ".join([f"params.{k} = '{v}'" for k, v in expanded_grid[i].items()]))
  train_loader = DataLoader(train_dataset, batch_size=expanded_grid[i]['batch_size'], shuffle=True)
    
  test_loader = DataLoader(test_dataset, batch_size=expanded_grid[i]['batch_size'], shuffle=False)

  if len(runs) == 0:


    best_loss=100
    best_model_train_acc=0
    best_model_test_acc=0
    best_model_test_loss=0
    best_model_train_loss=0
    min_delta=0
    patience=3
    model = MyModel(num_classes = 7)

    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.SGD(model.parameters(), lr=expanded_grid[i]['lr'], momentum =expanded_grid[i]['momentum'])


    mlflow.start_run()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    num_epochs = 100
    acc_list_train=[]
    acc_list_test=[]    
    model.train()

    mlflow.set_tag("model_name", "ResNet50")
    mlflow.log_param("lr", expanded_grid[i]['lr'])
    mlflow.log_param("momentum", expanded_grid[i]['momentum'])
    mlflow.log_param("batch_size", expanded_grid[i]['batch_size'])
    mlflow.log_param("decay", expanded_grid[i]['decay'])
    best_loss = 100
    counter=0
    stop=False
    for epoch in range(num_epochs):  
        if stop:
          break
        
        running_loss = 0.0
        running_acc = 0.0
        tot_seen = 0
        for X_batch, y_batch in train_loader:
            X_batch = torch.reshape(X_batch, (X_batch.shape[0], 1, 24, 24, 24))
            X_batch = X_batch.type(torch.cuda.FloatTensor)
            #manda i batch al device 
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            #azzera gradiente
            optimizer.zero_grad()
            #predict
            y_pred = model(X_batch)
            

            #print(y_batch.shape)
            loss = criterion(y_pred, y_batch)
            _,acc = accuracy(y_pred, y_batch)
            #backpropagation della loss
            loss.backward()
            #ottimizzazione
            optimizer.step()
            
            #somma della loss e dell'accuracy per il batch
            running_loss += loss.item()
            running_acc += acc
            tot_seen += len(y_batch)                    
            
        
        print(f'Epoch {epoch}: | Loss: {running_loss/len(train_loader):.5f} | Acc: {running_acc/tot_seen:.3f}')
        acc_list_train.append(running_acc/len(train_loader))
        mlflow.log_metric("train_loss", running_loss / len(train_loader), step=epoch)
        mlflow.log_metric("train_acc", running_acc/tot_seen, step=epoch)
        tot_corrette = 0
        tot_eseguite = 0
        running_test_loss = 0
        val_loss = 0
        
        with torch.no_grad():

          for images, labels in test_loader:
              images = torch.reshape(images, (images.shape[0], 1, 24, 24, 24))
              images = images.type(torch.cuda.FloatTensor)
              images = images.to(device)
              labels = labels.to(device)
            
              outputs = model(images)
              test_loss = criterion(outputs, labels)
              _, n_corrette=accuracy(outputs, labels)
              
              running_test_loss += test_loss.item() 
              tot_corrette+=n_corrette.item()
              tot_eseguite+=labels.shape[0]

          test_acc=100* (tot_corrette/tot_eseguite)
          val_loss = running_test_loss / len(test_loader)
          acc_list_test.append(test_acc)
          print("Test acc: ", test_acc)
          print("Test loss: ", val_loss)
          
          mlflow.log_metric("test_acc", test_acc, step=epoch)
          mlflow.log_metric("test_loss", val_loss, step=epoch)
          
          
        if val_loss < best_loss - min_delta:
          print("MIGLIORATO")
          best_loss = val_loss
          best_model_train_acc=running_acc/tot_seen
          best_model_test_acc=test_acc
          best_model_test_loss=val_loss
          best_model_train_loss=running_loss / len(train_loader)
          counter = 0
          # Salva i pesi del modello se la validation loss è migliorata
          torch.save(model.state_dict(), 'best_model.pt')
        else:
          counter += 1
        # Verifica se raggiunto il criterio di early stopping
          if counter >= patience:
              print(f'Early stopping at epoch {epoch+1}')
              stop=True
        print("BEST TEST LOSS: ", best_loss)

    mlflow.set_tag("Epochs_stopped", epoch+1)
    mlflow.log_artifact("best_model.pt")
    mlflow.log_metric("best_test_acc", best_model_test_acc)
    mlflow.log_metric("best_test_loss", best_model_test_loss)
    mlflow.log_metric("best_train_acc", best_model_train_acc)
    mlflow.log_metric("best_train_loss", best_model_train_loss)
    mlflow.end_run()

    print('Finished Training')

    PATH = './cnn.pth'
    torch.save(model.state_dict(), PATH)
    torch.cuda.empty_cache()
  else:
    print("RUN: ", [f"params.{k} = '{v}'" for k, v in expanded_grid[i].items()], " già completata" )
    


Epoch 0: | Loss: 1.80042 | Acc: 0.267
Test acc:  27.067259689570662
Test loss:  1.7811005377356028
MIGLIORATO
BEST TEST LOSS:  1.7811005377356028
Epoch 1: | Loss: 1.75978 | Acc: 0.291
Test acc:  30.289552509757645
Test loss:  1.732993905254871
MIGLIORATO
BEST TEST LOSS:  1.732993905254871
Epoch 2: | Loss: 1.71957 | Acc: 0.312
Test acc:  29.754016519923752
Test loss:  1.755112903655609
BEST TEST LOSS:  1.732993905254871
Epoch 3: | Loss: 1.69195 | Acc: 0.330
Test acc:  32.90369429064173
Test loss:  1.700087672024104
MIGLIORATO
BEST TEST LOSS:  1.700087672024104
Epoch 4: | Loss: 1.67311 | Acc: 0.341
Test acc:  33.66615231006626
Test loss:  1.6852704582875864
MIGLIORATO
BEST TEST LOSS:  1.6852704582875864
Epoch 5: | Loss: 1.65020 | Acc: 0.350
Test acc:  36.00798765544159
Test loss:  1.6449159941921345
MIGLIORATO
BEST TEST LOSS:  1.6449159941921345
Epoch 6: | Loss: 1.63093 | Acc: 0.362
Test acc:  36.03521829899247
Test loss:  1.6498729596937323
BEST TEST LOSS:  1.6449159941921345
Epoch 7: |